In [23]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
from astronomaly.data_management.image_reader import AstroImage
from astronomaly.data_management import image_reader
from astronomaly.data_management.image_reader import ImageDataset
from astronomaly.preprocessing import image_preprocessing
from astronomaly.feature_extraction import shape_features
from astronomaly.postprocessing import scaling
from astronomaly.anomaly_detection import isolation_forest, human_loop_learning
from astronomaly.visualisation import tsne

In [25]:
def apply_transform(cutout, transform_function):
    """
    Applies the transform function(s) given at initialisation to the image.

    Parameters
    ----------
    cutout : np.ndarray
        Cutout of image

    Returns
    -------
    np.ndarray
        Transformed cutout
    """
    if transform_function is not None:
        try:
            len(transform_function)
            new_cutout = cutout
            for f in transform_function:
                new_cutout = f(new_cutout)
            cutout = new_cutout
        except TypeError:  # Simple way to test if there's only one function
            cutout = transform_function(cutout)
    return cutout

_____

### Basic setting for DECaLS data

In [26]:
which_data = 'decals'

coadd_id = '026'

feature_method = 'ellipse'

dim_reduction = ''

band_prefixes = ['z-', 'r-', 'g-']

band_rgb = {'r': 'z-', 'g': 'r-', 'b': 'g-'}

image_transform_function = [image_preprocessing.image_transform_sigma_clipping,
                            # image_preprocessing.image_transform_inverse_sinh,
                            image_preprocessing.image_transform_scale
                            ]

display_transform_function = [#image_preprocessing.image_transform_inverse_sinh,
                              image_preprocessing.image_transform_scale]

plot_cmap = 'hot'
window_size = 32
list_of_files = []

### Reading in the catalogue

In [27]:
data_dir = '/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/'
#data_dir = '/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/'

image_dir = os.path.join(data_dir, 'Images')

output_dir = os.path.join(
    '/home/verlon/Desktop/Astronomaly/Data/Output/Coadd026','')

#output_dir = os.path.join(
#    '/home/verlon/Desktop/Astronomaly/Data/Output/0267 Brick', '')


#catalogue = pd.read_csv('/home/verlon/Desktop/Astronomaly/Data/test_catalogue.csv')
catalogue = pd.read_csv('/home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/test_catalogue_0260m062_500.csv')
#catalogue = pd.read_csv('/home/verlon/Desktop/Astronomaly/Data/Input/0267 Brick/Catalogue/large_catalogue.csv')

### Running Astronomaly

In [28]:
image_dataset = image_reader.ImageDataset(directory=image_dir,
                                          list_of_files=list_of_files,
                                          window_size=window_size, 
                                          output_dir=output_dir, 
                                          plot_square=False,
                                          transform_function=image_transform_function,
                                          display_transform_function=display_transform_function,
                                          plot_cmap=plot_cmap,
                                          catalogue=catalogue,
                                          band_prefixes=band_prefixes,
                                          bands_rgb=band_rgb)

Reading image data from /home/verlon/Desktop/Astronomaly/Data/Input/Coadd_0260/0260m062/Images/z-legacysurvey-0260m062-image.fits.fz...


2020-10-05 15:53:23,280 - WARNING - FITSFixedWarning: 'datfix' made the change 'Set DATE-REF to '1858-11-17' from MJD-REF'.


Done!
A catalogue of  500 sources has been provided.


### After adding 'peak_flux' column (copying flux_g column)

In [29]:
image_dataset.catalogue

,Unnamed: 0,objid,original_image,flux_g,flux_r,flux_z,x,y,ra,dec,type,peak_flux
0,594,594,legacysurvey-0260m062-image.fits.fz,317.535095,720.682556,1307.191528,337,369,26.126581,-6.354089,DEV,NaN
1,6095,6095,legacysurvey-0260m062-image.fits.fz,168.212524,284.601593,412.424927,994,2486,26.078486,-6.199997,EXP,NaN
2,2709,2709,legacysurvey-0260m062-image.fits.fz,51.362320,146.547363,276.822815,3216,1209,25.915781,-6.292911,DEV,NaN
3,5667,5667,legacysurvey-0260m062-image.fits.fz,71.172943,143.411240,247.036163,1088,2359,26.071583,-6.209265,EXP,NaN
4,2471,2471,legacysurvey-0260m062-image.fits.fz,43.115738,121.118385,238.656158,2859,1137,25.941948,-6.298205,DEV,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
495,6345,6345,legacysurvey-0260m062-image.fits.fz,0.981929,2.535009,4.758138,292,2585,26.129869,-6.192759,REX,NaN
496,4612,4612,legacysurvey-0260m062-image.fits.fz,1.468138,3.162834,4.752346,3450,1864,25.898683,-6.245235,EXP,NaN
497,2445,2445,legacysurvey-0260m062-image.fits.fz,0.865858,2.835724,4.745888,1230,1161,26.061230,-6.296397,DEV,NaN
498,3190,3190,legacysurvey-0260m062-image.fits.fz,0.940797,2.619087,4.742189,2916,1365,25.937799,-6.281606,EXP,NaN


In [30]:
image_dataset.metadata.peak_flux = image_dataset.catalogue.flux_g

In [31]:
image_dataset.metadata

,original_image,x,y,ra,dec,peak_flux
594,legacysurvey-0260m062-image.fits.fz,337,369,26.126581,-6.354089,NaN
6095,legacysurvey-0260m062-image.fits.fz,994,2486,26.078486,-6.199997,NaN
2709,legacysurvey-0260m062-image.fits.fz,3216,1209,25.915781,-6.292911,NaN
5667,legacysurvey-0260m062-image.fits.fz,1088,2359,26.071583,-6.209265,NaN
2471,legacysurvey-0260m062-image.fits.fz,2859,1137,25.941948,-6.298205,NaN
...,...,...,...,...,...,...
6345,legacysurvey-0260m062-image.fits.fz,292,2585,26.129869,-6.192759,NaN
4612,legacysurvey-0260m062-image.fits.fz,3450,1864,25.898683,-6.245235,NaN
2445,legacysurvey-0260m062-image.fits.fz,1230,1161,26.061230,-6.296397,NaN
3190,legacysurvey-0260m062-image.fits.fz,2916,1365,25.937799,-6.281606,NaN


image_dataset.get_sample('62763') #Just zero values

inds = ['62762', '62763', '67877']
imgs = []

for i in inds:
    imgs.append(image_dataset.get_sample(i))

In [32]:
imgs

[]

In [33]:
pipeline_ellipse = shape_features.EllipseFitFeatures(
    percentiles=[90, 80, 70, 60, 50, 0],
    output_dir=output_dir, channel=0, force_rerun=True
)

features_original = pipeline_ellipse.run_on_dataset(image_dataset)

Extracting features using EllipseFitFeatures ...
0 instances completed
100 instances completed
200 instances completed
300 instances completed
400 instances completed
Done! Time taken:  8.213062763214111 s


In [34]:
features = features_original.copy()

In [35]:
pipeline_scaler = scaling.FeatureScaler(force_rerun=True,
                                            output_dir=output_dir)

In [36]:
features = pipeline_scaler.run(features)

Running FeatureScaler ...
Done! Time taken: 0.042346954345703125 s


In [37]:
pipeline_iforest = isolation_forest.IforestAlgorithm(
    force_rerun=True, output_dir=output_dir)
anomalies = pipeline_iforest.run(features)

pipeline_score_converter = human_loop_learning.ScoreConverter(
    force_rerun=False, output_dir=output_dir)
anomalies = pipeline_score_converter.run(anomalies)
anomalies = anomalies.sort_values('score', ascending=False)

Running IforestAlgorithm ...
Done! Time taken: 0.686532735824585 s
Running ScoreConverter ...
Running anomaly score rescaler...
Done! Time taken: 0.01149892807006836 s


In [38]:
try:
    df = pd.read_csv(
        os.path.join(output_dir, 'ml_scores.csv'), 
        index_col=0,
        dtype={'human_label': 'int'})
    df.index = df.index.astype('str')

    if len(anomalies) == len(df):
        anomalies = pd.concat(
            (anomalies, df['human_label']), axis=1, join='inner')
except FileNotFoundError:
    pass
    
pipeline_active_learning = human_loop_learning.NeighbourScore(
    alpha=1, output_dir=output_dir)

pipeline_tsne = tsne.TSNE_Plot(
    force_rerun=False,
    output_dir=output_dir,
    perplexity=50)
t_plot = pipeline_tsne.run(features.loc[anomalies.index])

Running TSNE_Plot ...
Done! Time taken: 5.126437187194824 s
